In [1]:
%load_ext autoreload
%autoreload 2
import datetime
import numpy as np
import os
import pandas as pd
import sys

project_dir = os.sep.join([os.getcwd(), '..'])
#sys.path.append(os.sep.join([project_dir, 'src', 'processed']))
processed_data_dir = os.sep.join([project_dir, 'data', 'processed'])

%matplotlib inline

In [2]:
df_date = pd.read_feather(os.sep.join([processed_data_dir, 'datetime_features.feather']))
df_met  = pd.read_feather(os.sep.join([processed_data_dir, 'blindern_met_features.feather']))
df_sun  = pd.read_feather(os.sep.join([processed_data_dir, 'solar_elevation_1h.feather']))
dfs = [df_date, df_met, df_sun]

In [3]:
for df in dfs:
    display(df.head())

,DateTime,hour,weekday,month,public_holiday,school_holiday
0,2016-04-01 00:00:00+00:00,0,4,4,0,0
1,2016-04-01 00:05:00+00:00,0,4,4,0,0
2,2016-04-01 00:10:00+00:00,0,4,4,0,0
3,2016-04-01 00:15:00+00:00,0,4,4,0,0
4,2016-04-01 00:20:00+00:00,0,4,4,0,0


,level_0,index,temperature,wind_speed,humidity,sunshine,precipitation,wind_direction_cat,weather_fair/cloudy,weather_fog/haze,weather_thunderstorm,weather_rain,weather_snow,weather_other
0,0,2016-04-01 00:00:00+00:00,2.1,6.0,46.0,0.0,0.0,7,1,0,0,0,0,0
1,1,2016-04-01 01:00:00+00:00,2.2,5.7,45.0,0.0,0.0,0,1,0,0,0,0,0
2,2,2016-04-01 02:00:00+00:00,0.7,5.8,51.0,0.0,0.0,0,1,0,0,0,0,0
3,3,2016-04-01 03:00:00+00:00,-0.3,2.0,55.0,0.0,0.0,1,1,0,0,0,0,0
4,4,2016-04-01 04:00:00+00:00,-1.8,2.2,62.0,0.0,0.0,5,1,0,0,0,0,0


,index,solar_elevation_angle,solar_elevation_cat
0,2016-04-01 00:00:00+00:00,-25.016272,0
1,2016-04-01 01:00:00+00:00,-22.599384,0
2,2016-04-01 02:00:00+00:00,-18.344495,0
3,2016-04-01 03:00:00+00:00,-12.646478,0
4,2016-04-01 04:00:00+00:00,-5.935906,0


In [4]:
df_joined = pd.merge(df_date, df_met, left_on='DateTime', right_on='index', how='left')
df_joined.drop(columns=['level_0', 'index'], inplace=True)
df_joined = pd.merge(df_joined, df_sun, left_on='DateTime', right_on='index', how='left')
df_joined.drop(columns=['index', 'solar_elevation_cat'], inplace=True)

In [5]:
all_vars = df_joined.columns
cont_vars = ['DateTime', 'temperature', 'wind_speed', 'humidity', 'sunshine', 'precipitation', 'solar_elevation_angle']
cat_vars = [var for var in all_vars if not var in cont_vars]

In [6]:
# A lot of the rows are missing data in the met columns. This seems to be due to missing observations, mainly (but not all) at night. See also further below.
for v in cat_vars:
    print(v, df_joined[v].isna().sum())

hour 0
weekday 0
month 0
public_holiday 0
school_holiday 0
wind_direction_cat 237231
weather_fair/cloudy 237231
weather_fog/haze 237231
weather_thunderstorm 237231
weather_rain 237231
weather_snow 237231
weather_other 237231


In [7]:
for var in ['temperature', 'wind_speed', 'humidity', 'solar_elevation_angle']:
    df_joined[var].interpolate(method='linear', inplace=True)
for var in ['wind_direction_cat', 'sunshine', 'precipitation', 'weather_fair/cloudy', 'weather_fog/haze', 'weather_thunderstorm', 'weather_rain', 'weather_snow', 'weather_other']:
    df_joined[var].fillna(method='ffill', limit=11, inplace=True)

In [8]:
df_joined.head(13)

,DateTime,hour,weekday,month,public_holiday,school_holiday,temperature,wind_speed,humidity,sunshine,precipitation,wind_direction_cat,weather_fair/cloudy,weather_fog/haze,weather_thunderstorm,weather_rain,weather_snow,weather_other,solar_elevation_angle
0,2016-04-01 00:00:00+00:00,0,4,4,0,0,2.100000,6.000,46.000000,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-25.016272
1,2016-04-01 00:05:00+00:00,0,4,4,0,0,2.108333,5.975,45.916667,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.814865
2,2016-04-01 00:10:00+00:00,0,4,4,0,0,2.116667,5.950,45.833333,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.613458
3,2016-04-01 00:15:00+00:00,0,4,4,0,0,2.125000,5.925,45.750000,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.412050
4,2016-04-01 00:20:00+00:00,0,4,4,0,0,2.133333,5.900,45.666667,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.210643
5,2016-04-01 00:25:00+00:00,0,4,4,0,0,2.141667,5.875,45.583333,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.009236
6,2016-04-01 00:30:00+00:00,0,4,4,0,0,2.150000,5.850,45.500000,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-23.807828
7,2016-04-01 00:35:00+00:00,0,4,4,0,0,2.158333,5.825,45.416667,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-23.606421
8,2016-04-01 00:40:00+00:00,0,4,4,0,0,2.166667,5.800,45.333333,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-23.405014
9,2016-04-01 00:45:00+00:00,0,4,4,0,0,2.175000,5.775,45.250000,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-23.203606


In [34]:
for v in cat_vars:
    print(v, df_joined[v].isna().sum())
    #df_joined[v] = df_joined[v].astype('int32')      # This does not work because there are nans in the columns.
    #df_joined[v] = df_joined[v].astype('category')   # So we have to transform to categories when we have dropped the nan-rows.
#print(df_joined.loc[df_joined['wind_direction_cat'].isna(), :].head(20))

hour 0
weekday 0
month 0
public_holiday 0
school_holiday 0
wind_direction_cat 49417
weather_fair/cloudy 49417
weather_fog/haze 49417
weather_thunderstorm 49417
weather_rain 49417
weather_snow 49417
weather_other 49417


In [35]:
df_joined.head(13)

,DateTime,hour,weekday,month,public_holiday,school_holiday,temperature,wind_speed,humidity,sunshine,precipitation,wind_direction_cat,weather_fair/cloudy,weather_fog/haze,weather_thunderstorm,weather_rain,weather_snow,weather_other,solar_elevation_angle
0,2016-04-01 00:00:00+00:00,0,4,4,0,0,2.100000,6.000,46.000000,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-25.016272
1,2016-04-01 00:05:00+00:00,0,4,4,0,0,2.108333,5.975,45.916667,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.814865
2,2016-04-01 00:10:00+00:00,0,4,4,0,0,2.116667,5.950,45.833333,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.613458
3,2016-04-01 00:15:00+00:00,0,4,4,0,0,2.125000,5.925,45.750000,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.412050
4,2016-04-01 00:20:00+00:00,0,4,4,0,0,2.133333,5.900,45.666667,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.210643
5,2016-04-01 00:25:00+00:00,0,4,4,0,0,2.141667,5.875,45.583333,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-24.009236
6,2016-04-01 00:30:00+00:00,0,4,4,0,0,2.150000,5.850,45.500000,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-23.807828
7,2016-04-01 00:35:00+00:00,0,4,4,0,0,2.158333,5.825,45.416667,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-23.606421
8,2016-04-01 00:40:00+00:00,0,4,4,0,0,2.166667,5.800,45.333333,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-23.405014
9,2016-04-01 00:45:00+00:00,0,4,4,0,0,2.175000,5.775,45.250000,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,-23.203606


In [ ]:
# The following can be used when we want to mask away the hours when the stations are closed:

In [30]:
mask = (df_met['index'].dt.hour >= 10) & (df_met['index'].dt.hour <= 11)

In [31]:
mask

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10        True
11        True
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
17044    False
17045    False
17046    False
17047    False
17048    False
17049    False
17050    False
17051    False
17052    False
17053    False
17054    False
17055    False
17056    False
17057    False
17058    False
17059    False
17060     True
17061     True
17062    False
17063    False
17064    False
17065    False
17066    False
17067    False
17068    False
17069    False
17070    False
17071    False
17072    False
17073    False
Name: index, Length: 17074, dtype: bool